In [97]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [98]:
df=pd.read_csv("fake_train.csv")
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [99]:
##Dropping the nan values
df=df.dropna()

In [100]:
#Get the independent features
X=df.iloc[:,:-1]
#Get the dependent features
y=df.iloc[:,-1]

In [101]:
X.shape

(18285, 4)

In [102]:
y.shape

(18285,)

In [103]:
import tensorflow as tf
tf.__version__

'2.4.1'

In [104]:
from tensorflow.keras.layers import Embedding
#to keep our input size fixed(sentences of equal length)
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [105]:
voc_size=5000

### One Hot Representation

In [106]:
messages=X.copy()

In [107]:
#As we have dropped values from te dataset so we are resetting index
messages.reset_index(inplace=True)

In [108]:
import nltk
import re
from nltk.corpus import stopwords

In [109]:
#stopwords are the words that are not important
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\icons\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [110]:
#Data preprocessing 
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
corpus=[]
for i in range(len(messages)):
    #Subtitute everything other than words from a to z and A to Z with a space
    review=re.sub("[^a-zA-Z]"," ",messages["title"][i])
    #covert everything in lowercase 
    review=review.lower()
    review=review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review=" ".join(review)
    corpus.append(review)

In [111]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [112]:
#we will do one hot representation of corpus. It is giving index to every word according to vocabulary size
onehot=[one_hot(words,voc_size) for words in corpus]
onehot

[[2819, 3658, 1154, 1914, 635, 4936, 3480, 3997, 2428, 1290],
 [2056, 1683, 2990, 2975, 3123, 2459, 78],
 [2245, 3166, 2652, 751],
 [4056, 1417, 2357, 3149, 1028, 1992],
 [1227, 3123, 3513, 1370, 1702, 4611, 3123, 2802, 1968, 2699],
 [1027,
  1507,
  4820,
  4105,
  789,
  1736,
  2694,
  4824,
  3376,
  1759,
  2930,
  2250,
  2591,
  3034,
  78],
 [1031, 2916, 4199, 4948, 3150, 1924, 4016, 325, 4192, 323, 3922],
 [4402, 3301, 3792, 4861, 1864, 4553, 1736, 4774, 4192, 323, 3922],
 [844, 3753, 4651, 1907, 1622, 1586, 907, 1253, 1736, 2343],
 [258, 2264, 3920, 2395, 2323, 1210, 4921, 3197],
 [839, 587, 1934, 4206, 35, 2273, 3751, 2088, 1220, 1179, 3823],
 [3149, 2922, 635, 1586, 1736, 1864],
 [1669, 3475, 4795, 3325, 3079, 4051, 3534, 342, 2849],
 [3881, 4226, 1607, 2883, 1947, 2219, 4114, 4192, 323, 3922],
 [1492, 3267, 647, 2614, 2485, 4192, 323, 3922],
 [1609, 4101, 895, 2205, 2529, 787, 4618, 4500, 2778, 2213],
 [518, 1200, 1683],
 [1373, 3202, 3162, 3121, 1736, 2671, 3122, 78],
 [1

### Embedding Representation

In [113]:
sent_len=20
#if the senntence length is not 20 then it will add 0 in front of the sentence to make its length 20
embedded_docs=pad_sequences(onehot,padding="pre",maxlen=sent_len)
embedded_docs

array([[   0,    0,    0, ..., 3997, 2428, 1290],
       [   0,    0,    0, ..., 3123, 2459,   78],
       [   0,    0,    0, ..., 3166, 2652,  751],
       ...,
       [   0,    0,    0, ..., 4192,  323, 3922],
       [   0,    0,    0, ..., 2235, 3612, 4060],
       [   0,    0,    0, ..., 4111, 3720,  130]])

In [114]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 2819,
       3658, 1154, 1914,  635, 4936, 3480, 3997, 2428, 1290])

In [115]:
len(embedded_docs)

18285

## Creating Model

In [116]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_len))
model.add(LSTM(100))
model.add(Dense(1,activation="sigmoid"))
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

In [117]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
lstm_6 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [118]:
len(embedded_docs),y.shape

(18285, (18285,))

In [119]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [120]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [121]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [122]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 12s 43ms/step - loss: 0.4761 - accuracy: 0.7553 - val_loss: 0.1869 - val_accuracy: 0.9210
Epoch 2/10
192/192 [==============================] - 7s 39ms/step - loss: 0.1356 - accuracy: 0.9472 - val_loss: 0.1978 - val_accuracy: 0.9211
Epoch 3/10
192/192 [==============================] - 6s 34ms/step - loss: 0.0871 - accuracy: 0.9700 - val_loss: 0.2124 - val_accuracy: 0.9120
Epoch 4/10
192/192 [==============================] - 6s 33ms/step - loss: 0.0574 - accuracy: 0.9818 - val_loss: 0.2491 - val_accuracy: 0.9148
Epoch 5/10
192/192 [==============================] - 7s 35ms/step - loss: 0.0392 - accuracy: 0.9878 - val_loss: 0.2945 - val_accuracy: 0.9138
Epoch 6/10
192/192 [==============================] - 8s 41ms/step - loss: 0.0255 - accuracy: 0.9927 - val_loss: 0.3687 - val_accuracy: 0.9120
Epoch 7/10
192/192 [==============================] - 7s 37ms/step - loss: 0.0232 - accuracy: 0.9918 - val_loss: 0.3653 - val_accuracy: 0.905

### Performance metrics and accuracy

In [124]:
y_pred=model.predict_classes(X_test)

C:\Users\icons\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [125]:
from sklearn.metrics import confusion_matrix

In [126]:
confusion_matrix(y_test,y_pred)

array([[3107,  312],
       [ 234, 2382]], dtype=int64)

In [127]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9095277547638774

we got an accuracy of 90% with just one layer which is pretty good.